# Odległość edycyjna

In [5]:
# miacierz odlgłości edycyjnej - tablica w której dynamicznie zapisujemy 
# odległość edycyjną między prefiksami słów "s" i "t"
def levenshtein_distance_matrix(s, t):
    m, n = len(s), len(t)
    # inicjalizacja macierzy
    d = [[0] * (n + 1) for i in range(m + 1)]
    for i in range(m + 1):
        d[i][0] = i
    for j in range(n + 1):
        d[0][j] = j
    # wypełnienie macierzy
    for j in range(1, n + 1):
        for i in range(1, m + 1):
            if s[i - 1] == t[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                d[i][j] = min(d[i - 1][j], d[i][j - 1], d[i - 1][j - 1]) + 1
    return d

# odległość edycyjna między "s" i "t"
def levenshtein_distance(s, t):
    m, n = len(s), len(t)
    return levenshtein_distance_matrix(s,t)[m][n]


## Wizualizacja

In [52]:
from enum import Enum

# 4 typy rodzaju edycji tekstu (z czego jeden trywialny: NONE)
class EditType(Enum):
    NONE = 0
    INSERT = 1
    DELETE = 2
    REPLACE = 3

# wszystkie zmiany prowadzące ze słowa "s" na "t"
def transwer_word_edits(s,t):
    d = levenshtein_distance_matrix(s,t)
    i, j = len(s), len(t) 
    edits = []
    while i > 0 or j > 0:
        if i > 0 and j > 0 and d[i - 1][j - 1] == d[i][j] and s[i - 1] == t[j - 1]:
            i, j = i - 1, j - 1
            edits.append((EditType.NONE, s[i]))
        elif i > 0 and d[i - 1][j] == d[i][j] - 1:
            i -= 1
            edits.append((EditType.DELETE, s[i]))
        elif j > 0 and d[i][j - 1] == d[i][j] - 1:
            j -= 1
            edits.append((EditType.INSERT, t[j]))
        elif i > 0 and j > 0 and d[i - 1][j - 1] == d[i][j] - 1:
            i, j = i - 1, j - 1
            edits.append((EditType.REPLACE, (s[i], t[j])))
    return edits[::-1]

# wizualizacja od lewej do prawej 
def transwer_word_wizualation(s, t):
    edits = transwer_word_edits(s,t)
    curr_s = s[:]+" "
    i=0
    for edType, edLett in edits:
        if edType == EditType.NONE:
            pass            
        elif edType == EditType.DELETE:
            print(f"{s} --(Delete letter '{edLett}' at index {i})--> {s[:i] + s[i+1:]}")
            s = s[:i] + s[i+1:]
            i -= 1
        elif edType == EditType.INSERT:
            print(f"{s} --(Inserted letter '{edLett}' at index {i})--> {s[:i] + edLett + s[i:]}")
            s = s[:i] + edLett + s[i:]
        elif edType == EditType.REPLACE:
            print(f"{s} --('{edLett[0]}' Replace with '{edLett[1]}' at index {i})--> {s[:i] + edLett[1] + s[i+1:]}")
            s = s[:i] + edLett[1] + s[i+1:]
        i += 1


przykładowe wywołania

In [53]:
s1,s2 = "los", "kloc"
print(f"{s1} --> {s2} in {levenshtein_distance(s1, s2)} steps")
transwer_word_wizualation(s1, s2)
print()

s1,s2 = "Łódź", "Lodz"
print(f"{s1} --> {s2} in {levenshtein_distance(s1, s2)} steps")
transwer_word_wizualation(s1, s2)
print()

s1,s2 = "kwintesencja", "quintessence"
print(f"{s1} --> {s2} in {levenshtein_distance(s1, s2)} steps")
transwer_word_wizualation(s1, s2)
print()

s1,s2 = "ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG"
print(f"{s1} --> {s2} in {levenshtein_distance(s1, s2)} steps")
transwer_word_wizualation(s1, s2)
print()


los --> kloc in 2 steps
los --(Inserted letter 'k' at index 0)--> klos
klos --('s' Replace with 'c' at index 3)--> kloc

Łódź --> Lodz in 3 steps
Łódź --('Ł' Replace with 'L' at index 0)--> Lódź
Lódź --('ó' Replace with 'o' at index 1)--> Lodź
Lodź --('ź' Replace with 'z' at index 3)--> Lodz

kwintesencja --> quintessence in 5 steps
kwintesencja --('k' Replace with 'q' at index 0)--> qwintesencja
qwintesencja --('w' Replace with 'u' at index 1)--> quintesencja
quintesencja --(Inserted letter 's' at index 6)--> quintessencja
quintessencja --('j' Replace with 'e' at index 11)--> quintessencea
quintessencea --(Delete letter 'a' at index 12)--> quintessence

ATGAATCTTACCGCCTCG --> ATGAGGCTCTGGCCCCTG in 7 steps
ATGAATCTTACCGCCTCG --('A' Replace with 'G' at index 4)--> ATGAGTCTTACCGCCTCG
ATGAGTCTTACCGCCTCG --('T' Replace with 'G' at index 5)--> ATGAGGCTTACCGCCTCG
ATGAGGCTTACCGCCTCG --(Inserted letter 'C' at index 8)--> ATGAGGCTCTACCGCCTCG
ATGAGGCTCTACCGCCTCG --('A' Replace with 'G' at index 

# Najdłuższy wspólny podciąg
### $\color{grey}{\text{(Longest Common Subseqence)}}$

In [100]:
# miacierz najdłuższego wspólnego podciągu - tablica w której dynamicznie zapisujemy 
# dlóugość najdłuższego wspólnego podciągu między prefiksami słów "s" i "t"
def lcs_matrix(s, t, key=lambda x:x):
    m, n = len(s), len(t)
    # inicjalizacja macierzy
    d = [[0] * (n + 1) for i in range(m + 1)]
    # wypłnianie wartościami
    for j in range(1,n+1):
        for i in range(1,m+1):
            if key(s[i-1]) == key(t[j-1]):
                d[i][j] = d[i-1][j-1] + 1
            else:
                d[i][j] = max(d[i][j-1], d[i-1][j])
    return d

#długość najdłurzszego wspólnego podciągu O(nm)
def lcs_len(s, t, key=lambda x:x):
    m, n = len(s), len(t)
    d = lcs_matrix(s, t, key)
    return d[m][n]

#najdłurzszy wspólny podciąg O(nm + min(m,n))
def lcs(s, t, key=lambda x:x):
    m, n = len(s), len(t)
    d = lcs_matrix(s, t, key)
    ans = ""
    prev = 0
    
    # wybór mniejszej petli
    if m < n:
        for i in range(m+1):
            if prev != d[i][n]:
                ans += key(s[i-1])
                prev = d[i][n]
    else:
        for i in range(n+1):
            if prev != d[m][i]:
                ans += key(t[i-1])
                prev = d[m][i]
    return ans

In [99]:
s1, s2 = "los", "kloc"
lcs(s1, s2)

'lo'

## podział pliku na tokeny

In [91]:
import spacy
import random
nlp = spacy.load("pl_core_news_sm") # załadowanie modelu językowego
origin_filename = "romeo-i-julia-700.txt"

# utworzenie obiektu Doc na podstawie tekstu z pliku filename
def create_doc(filename):
    with open(filename, "r") as f:
        text = f.read()

    return nlp(text)

# utworzenie nowego obiektu Doc bez 3% tokenów
def mangle_doc(doc):
    # wyznaczenie liczby tokenów do usunięcia
    num_tokens = len(doc)
    num_tokens_to_remove = int(0.03 * num_tokens)

    # wybranie losowych tokenów do usunięcia
    token_indices = list(range(num_tokens))
    random.shuffle(token_indices)
    tokens_to_remove = set(token_indices[:num_tokens_to_remove])

    return spacy.tokens.Doc(doc.vocab, words=[t.text for i, t in enumerate(doc) if i not in tokens_to_remove])

stworzenie dwóch obiektów DOC powstale po zniekształceniu tekstu wejściowego

In [113]:
doc_main = create_doc(origin_filename)

doc1 = mangle_doc(doc_main)
doc2 = mangle_doc(doc_main)

print(len(doc1))
print(doc1[0].text)
print(doc2[0].text)
print(doc1[0].text == doc2[0].text)

2619
William
William
True


In [114]:
# przykładowe wywołanie
lcs_len(doc1, doc2, lambda x:x.text)

2543

## Narzędzie diff

stworzenie plików do badania

In [121]:
def save_doc(doc, filename):
    new_text = doc.text # pobranie zmodyfikowanego tekstu
    with open(filename, "w") as f:
        f.write(new_text)

save_doc(doc1, "text1.txt")
save_doc(doc2, "text2.txt")
save_doc(doc_main, "text_main.txt")

In [117]:
def lcs_lines(lines1, lines2):
    m, n = len(lines1), len(lines2)
    d = lcs_matrix(lines1, lines2, lambda x:x)
    lcs_lines = []
    prev = 0

    # wybór mniejszej petli
    if m < n:
        for i in range(m+1):
            if prev != d[i][n]:
                lcs_lines.append(lines1[i-1])
                prev = d[i][n]
    else:
        for i in range(n+1):
            if prev != d[m][i]:
                lcs_lines.append(lines2[i-1])
                prev = d[m][i]
    return lcs_lines


def diff_lines(lines1, lines2):
    lcs = lcs_lines(lines1, lines2)

    result = []

    for i, line in enumerate(lines1):
        if line in lcs:
            continue
        else:
            result.append(("<", i+1, line))

    for i, line in enumerate(lines2):
        if line in lcs:
            continue
        else:
            result.append((">", i+1, line))

    result.sort(key=lambda x: x[1])

    return result


def diff_doc(doc1, doc2):
    lines1 = doc1.text.split("\n")
    lines2 = doc2.text.split("\n")
    return diff_lines(lines1, lines2)

In [115]:
diff_doc(doc1, doc2)

[('<', 6, ' ISBN 978 83 - 288 - 2903 - 9 '),
 ('>', 6, ' ISBN 978 - 83 - 288 - 2903 - 9 '),
 ('<', 10, ' OSOBY : * ESKALUS — książę panujący w Weronie '),
 ('>', 10, ' OSOBY : '),
 ('<', 11, '  * PARYS — młody Weroneńczyk szlachetnego rodu , księcia '),
 ('>', 11, '  * ESKALUS — książę panujący w Weronie '),
 ('>',
  12,
  '  * PARYS — młody Weroneńczyk szlachetnego rodu , krewny księcia '),
 ('<', 13, '  * STARZEC — stryjeczny brat Kapuleta '),
 ('>', 14, '  STARZEC — stryjeczny brat Kapuleta '),
 ('<', 16, '  * BENWOLIO — synowiec Montekiego '),
 ('>', 17, '  BENWOLIO — synowiec Montekiego '),
 ('<', 21, '  * SAMSON , GRZEGORZ — słudzy Kapuleta '),
 ('>', 22, '  * SAMSON , GRZEGORZ — Kapuleta '),
 ('<', 29, '  * KAPULET — małżonka Kapuleta '),
 ('>', 30, '  * PANI KAPULET małżonka Kapuleta '),
 ('<', 31, '  * MARTA — mamka Julii '),
 ('<',
  32,
  '  * Obywatele weroneńscy , różne osoby płci obojej , liczący się do przyjaciół obu domów , maski , straż wojskowa i inne osoby . '),
 ('>

## badanie dla zniekształconych plikow

In [119]:
def diff_files(filename1, filename2):
    with open(filename1, "r") as f:
        lines1 = f.read().split("\n")

    with open(filename2, "r") as f:
        lines2 = f.read().split("\n")
    return diff_lines(lines1, lines2)

In [122]:
diff_files("text_main.txt", "text1.txt")

[('<', 1, 'William Shakespeare'),
 ('>', 1, 'William Shakespeare '),
 ('<', 3, 'Romeo i Julia'),
 ('>', 3, ' Romeo i Julia '),
 ('<', 4, 'tłum. Józef Paszkowski'),
 ('>', 4, ' tłum . Józef Paszkowski '),
 ('<', 6, 'ISBN 978-83-288-2903-9'),
 ('>', 6, ' ISBN 978 83 - 288 - 2903 - 9 '),
 ('<', 10, 'OSOBY:'),
 ('>', 10, ' OSOBY : * ESKALUS — książę panujący w Weronie '),
 ('<', 11, ' * ESKALUS — książę panujący w Weronie'),
 ('>', 11, '  * PARYS — młody Weroneńczyk szlachetnego rodu , księcia '),
 ('<', 12, ' * PARYS — młody Weroneńczyk szlachetnego rodu, krewny księcia'),
 ('>',
  12,
  '  * MONTEKI , KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie '),
 ('<',
  13,
  ' * MONTEKI, KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie'),
 ('>', 13, '  * STARZEC — stryjeczny brat Kapuleta '),
 ('<', 14, ' * STARZEC — stryjeczny brat Kapuleta'),
 ('>', 14, '  * ROMEO — syn Montekiego '),
 ('<', 15, ' * ROMEO — syn Montekiego'),
 ('>', 15, '  * MERKUCJO — krewny księcia '),
 ('<', 16, '

In [123]:
diff_files("text_main.txt", "text2.txt")

[('<', 1, 'William Shakespeare'),
 ('>', 1, 'William Shakespeare '),
 ('<', 3, 'Romeo i Julia'),
 ('>', 3, ' Romeo i Julia '),
 ('<', 4, 'tłum. Józef Paszkowski'),
 ('>', 4, ' tłum . Józef Paszkowski '),
 ('<', 6, 'ISBN 978-83-288-2903-9'),
 ('>', 6, ' ISBN 978 - 83 - 288 - 2903 - 9 '),
 ('<', 10, 'OSOBY:'),
 ('>', 10, ' OSOBY : '),
 ('<', 11, ' * ESKALUS — książę panujący w Weronie'),
 ('>', 11, '  * ESKALUS — książę panujący w Weronie '),
 ('<', 12, ' * PARYS — młody Weroneńczyk szlachetnego rodu, krewny księcia'),
 ('>',
  12,
  '  * PARYS — młody Weroneńczyk szlachetnego rodu , krewny księcia '),
 ('<',
  13,
  ' * MONTEKI, KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie'),
 ('>',
  13,
  '  * MONTEKI , KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie '),
 ('<', 14, ' * STARZEC — stryjeczny brat Kapuleta'),
 ('>', 14, '  STARZEC — stryjeczny brat Kapuleta '),
 ('<', 15, ' * ROMEO — syn Montekiego'),
 ('>', 15, '  * ROMEO — syn Montekiego '),
 ('<', 16, ' * MERKUCJO — kre

In [124]:
diff_files("text2.txt", "text1.txt")

[('<', 6, ' ISBN 978 - 83 - 288 - 2903 - 9 '),
 ('>', 6, ' ISBN 978 83 - 288 - 2903 - 9 '),
 ('<', 10, ' OSOBY : '),
 ('>', 10, ' OSOBY : * ESKALUS — książę panujący w Weronie '),
 ('<', 11, '  * ESKALUS — książę panujący w Weronie '),
 ('>', 11, '  * PARYS — młody Weroneńczyk szlachetnego rodu , księcia '),
 ('<',
  12,
  '  * PARYS — młody Weroneńczyk szlachetnego rodu , krewny księcia '),
 ('>', 13, '  * STARZEC — stryjeczny brat Kapuleta '),
 ('<', 14, '  STARZEC — stryjeczny brat Kapuleta '),
 ('>', 16, '  * BENWOLIO — synowiec Montekiego '),
 ('<', 17, '  BENWOLIO — synowiec Montekiego '),
 ('>', 21, '  * SAMSON , GRZEGORZ — słudzy Kapuleta '),
 ('<', 22, '  * SAMSON , GRZEGORZ — Kapuleta '),
 ('>', 29, '  * KAPULET — małżonka Kapuleta '),
 ('<', 30, '  * PANI KAPULET małżonka Kapuleta '),
 ('>', 31, '  * MARTA — mamka Julii '),
 ('<',
  32,
  '  * MARTA — mamka Julii * Obywatele weroneńscy , różne osoby płci obojej , liczący się do przyjaciół obu domów , maski , straż wojskowa i